# **Week 2: Numerical Data Summaries**

```
.------------------------------------.
|   __  ____  ______  _  ___ _____   |
|  |  \/  \ \/ / __ )/ |/ _ \___  |  |
|  | |\/| |\  /|  _ \| | | | | / /   |
|  | |  | |/  \| |_) | | |_| |/ /    |
|  |_|  |_/_/\_\____/|_|\___//_/     |
'------------------------------------'

```


This week, we will explore basic summary statistics for numerical data and demonstrate how to compute them in R. If you are not familiar with R programming, please take some time to review [Week 0](https://colab.research.google.com/github/edelweiss611428/MXB107-Notebooks/blob/main/notebooks/Week_0.ipynb) and [Week 1](https://colab.research.google.com/github/edelweiss611428/MXB107-Notebooks/blob/main/notebooks/Week_1.ipynb) content.

## **Pre-Configurating the Notebook**

### **Switching to the R Kernel on Colab**

By default, Google Colab uses Python as its programming language. To use R instead, you’ll need to manually switch the kernel by going to **Runtime > Change runtime type**, and selecting R as the kernel. This allows you to run R code in the Colab environment.

However, our notebook is already configured to use R by default. Unless something goes wrong, you shouldn’t need to manually change runtime type.

### **Importing Required Datasets and Packages**
**Run the following lines of code**:

In [ ]:
#Do not modify

setwd("/content")

# Remove `MXB107-Notebooks` if exists,
if (dir.exists("MXB107-Notebooks")) {
  system("rm -rf MXB107-Notebooks")
}

# Fork the repository
system("git clone https://github.com/edelweiss611428/MXB107-Notebooks.git")

# Change working directory to "MXB107-Notebooks"
setwd("MXB107-Notebooks")

#
invisible(source("R/preConfigurated.R"))

**Do not modify the following**

In [ ]:
if (!require("testthat")) install.packages("testthat"); library("testthat")

test_that("Test if all packages have been loaded", {

  expect_true(all(c("ggplot2", "tidyr", "dplyr", "stringr", "magrittr", "IRdisplay", "png") %in% loadedNamespaces()))

})

test_that("Test if all utility functions have been loaded", {
  expect_true(exists("skewness"))
  expect_true(exists("kurtosis"))
  expect_true(exists("Mode"))
  expect_true(exists("FDbinning"))
  expect_true(exists("ModeBinMidpoint"))
  expect_true(exists("empiricalRuleGaussian"))
  expect_true(exists("chebyshevRule"))
  expect_true(exists("rangeBasedSD"))
  expect_true(exists("IQRBasedSD"))
  expect_true(exists("boxPlotDescribe"))
})

## **Workshop Questions**

### **Question 1**

The empirical rule can be extended beyond ±3 standard deviations.

| Interval | Approx. % of Data |
| -------- | ----------------- |
| ±1 SD    | 68.27%            |
| ±2 SD    | 95.45%            |
| ±3 SD    | 99.73%            |
| ±4 SD    | 99.994%           |
| ±5 SD    | 99.99994%         |
| ±6 SD    | 99.9999998%       |



The following code cell loads daily returns of `S&P 500` index from `03/01/2000` to `18/07/2025` from `s&p500.csv`, queried from Yahoo Finance.

In [ ]:
sp500_dailyReturn = 100*read.csv("datasets/s&p500.csv")$daily.returns

Use the `empiricalRuleGaussian()` function to visualise the empirical rule for up to ±6 standard deviations. Compare the empirical and theoretical coverage.

In [ ]:
empiricalRuleGaussian(sp500_dailyReturn, ks = 1:6)

# The empirical rule underestimates the probability of extreme return values, which can lead to an understatement of financial risk in worst-case scenarios.

### **EPA Fuel Economy Dataset**

A dataset containing information on over 13,500 cars sold in the US from 2010 to 2020, including measurements and characteristics related to vehicle fuel economy and specifications. Data sourced from the [US Fuel Economy website](https://www.fueleconomy.gov/feg/download.shtml).

| Variable | Description                                    |
|----------|------------------------------------------------|
| `city`   | EPA measured fuel economy in miles per gallon (city driving) |
| `hwy`    | EPA measured fuel economy in miles per gallon (highway driving) |
| `cyl`    | Number of cylinders in the engine              |
| `disp`   | Engine displacement (litres)                    |
| `drive`  | Vehicle drivetrain layout (e.g., FWD, RWD, AWD) |
| `make`   | Vehicle manufacturer name                       |
| `model`  | Vehicle model name                              |
| `trans`  | Transmission type (manual or automatic)        |
| `year`   | Vehicle model year                              |


In [ ]:
epa_data = read.csv("./datasets/epa_data.csv")
str(epa_data)

### **Question 2**

What is the mean and median city fuel economy for cars in the epa_data set? What is the mean, median, and mode for engine displacement?

**Hint**: Use the provided `Mode()` function. Use `na.rm = TRUE` option in `mean` and `median` to avoid errors caused by missing values. You may use the `summarise()` function for better readability.

In [ ]:
epa_data %>%
  summarise(mean_city_epa = mean(city, na.rm = TRUE),
            median_city_epa = median(city, na.rm = TRUE),
            mean_disp = mean(disp, na.rm = TRUE),
            median_disp = median(disp, na.rm = TRUE),
            mode_disp = Mode(disp))


### **Question 3**

Compute the variances and standard deviations for EPA city and highway mileage. Use Chebyshev’s theorem and the empirical rule to interpret the results. Is the empirical rule adequate?

**Hint**: Use the provided `empiricalRuleGaussian()` and `chebyshevRule()` functions. Use `na.rm = T` if needed.

In [ ]:
epa_data %>%
  summarise(var_city_epa = var(city, na.rm = T),
            sd_city_epa = sd(city, na.rm = T),
            var_hwy_epa = var(hwy, na.rm = T),
            sd_hwy_epa = sd(hwy, na.rm = T))


empiricalRuleGaussian(epa_data$city)
chebyshevRule(epa_data$city)
empiricalRuleGaussian(epa_data$hwy)
chebyshevRule(epa_data$hwy)

#Empirical rule is severely violated as data are positively skewed. Chebyshev rule is always satisfied but too conservative.


### **Question 4**

Compute the three quartiles and interquartile ranges for EPA city and highway mileage. Then, create side-by-side boxplots to visually compare city and highway mileage.

**Hint**: We may need a long format data frame to side-by-side boxplots.


In [ ]:
epa_data %>%
  summarise(
    city_Q1 = quantile(city, 0.25, na.rm = TRUE),
    city_Q2 = quantile(city, 0.50, na.rm = TRUE),
    city_Q3 = quantile(city, 0.75, na.rm = TRUE),
    city_IQR = IQR(city, na.rm = TRUE),

    hwy_Q1 = quantile(hwy, 0.25, na.rm = TRUE),
    hwy_Q2 = quantile(hwy, 0.50, na.rm = TRUE),
    hwy_Q3 = quantile(hwy, 0.75, na.rm = TRUE),
    hwy_IQR = IQR(hwy, na.rm = TRUE)
  )



epa_data %>%
  pivot_longer(cols = c(city, hwy),
               names_to = "type",
               values_to = "value") %>% #long-format df
  ggplot(aes(x = type, y = value, colour = type)) +
  geom_boxplot() +
  labs(title = "EPA: City driving vs. highway driving",
       x = "Type", y = "Miles per gallon") +
  theme_minimal() +
  theme(plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
        legend.position = "top")

